In [0]:
from pyspark import SparkContext


In [0]:

rdd = sc.textFile("/FileStore/tables/5000_Sales_Records.csv")

In [0]:
rdd.take(5)


Out[3]: ['Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit',
 'Central America and the Caribbean,Antigua and Barbuda ,Baby Food,Online,M,12/20/2013,957081544,1/11/2014,552,255.28,159.42,140914.56,87999.84,52914.72',
 'Central America and the Caribbean,Panama,Snacks,Offline,C,7/5/2010,301644504,7/26/2010,2167,152.58,97.44,330640.86,211152.48,119488.38',
 'Europe,Czech Republic,Beverages,Offline,C,9/12/2011,478051030,9/29/2011,4778,47.45,31.79,226716.10,151892.62,74823.48',
 'Asia,North Korea,Cereal,Offline,L,5/13/2010,892599952,6/15/2010,9016,205.70,117.11,1854591.20,1055863.76,798727.44']

In [0]:

rdd1= rdd.cache()

In [0]:
rdd11 = rdd1.map(lambda x : x.split(','))

In [0]:

header = rdd11.first()
rdd2 = rdd11.filter(lambda row: row != header)

In [0]:
rdd2.take(3)

Out[19]: [['Central America and the Caribbean',
  'Antigua and Barbuda ',
  'Baby Food',
  'Online',
  'M',
  '12/20/2013',
  '957081544',
  '1/11/2014',
  '552',
  '255.28',
  '159.42',
  '140914.56',
  '87999.84',
  '52914.72'],
 ['Central America and the Caribbean',
  'Panama',
  'Snacks',
  'Offline',
  'C',
  '7/5/2010',
  '301644504',
  '7/26/2010',
  '2167',
  '152.58',
  '97.44',
  '330640.86',
  '211152.48',
  '119488.38'],
 ['Europe',
  'Czech Republic',
  'Beverages',
  'Offline',
  'C',
  '9/12/2011',
  '478051030',
  '9/29/2011',
  '4778',
  '47.45',
  '31.79',
  '226716.10',
  '151892.62',
  '74823.48']]

In [0]:
rdd3=rdd2.map(lambda x:(x[1],1)).reduceByKey(lambda x,y:x+y)
rdd3.collect()

Out[11]: [('Antigua and Barbuda ', 26),
 ('North Korea', 24),
 ('Federated States of Micronesia', 20),
 ('Ethiopia', 26),
 ('Saint Lucia', 29),
 ('Lebanon', 31),
 ('Austria', 28),
 ('Mexico', 23),
 ('Trinidad and Tobago', 19),
 ('Libya', 32),
 ('Algeria', 28),
 ('Estonia', 23),
 ('Saudi Arabia', 23),
 ('Montenegro', 28),
 ('Guatemala', 28),
 ('Australia', 34),
 ('Malawi', 24),
 ('Somalia', 36),
 ('Switzerland', 28),
 ('Laos', 34),
 ('Angola', 16),
 ('Mauritania', 29),
 ('Finland', 23),
 ('Belgium', 26),
 ('Kiribati', 25),
 ('Uzbekistan', 27),
 ('South Korea', 36),
 ('Nigeria', 23),
 ('South Africa', 23),
 ('Netherlands', 24),
 ('Solomon Islands', 31),
 ('Iran', 29),
 ('Equatorial Guinea', 33),
 ('Iraq', 29),
 ('Mauritius ', 38),
 ('Eritrea', 19),
 ('Ukraine', 33),
 ('Myanmar', 33),
 ('Latvia', 27),
 ('Portugal', 25),
 ('Barbados', 26),
 ('Poland', 28),
 ('Zambia', 31),
 ('Slovenia', 27),
 ('Bhutan', 30),
 ('Cyprus', 30),
 ('Monaco', 13),
 ('Gabon', 22),
 ('Norway', 30),
 ('Thailand', 2

In [0]:
rdd33 = rdd3.repartition(1)

In [0]:

def toCSVLine(data):
   return ','.join(str(d) for d in data)

lines = rdd33.map(toCSVLine)
lines.saveAsTextFile('/FileStore/tables/output1.csv')
     

In [0]:
lines.take(5)

Out[14]: ['Antigua and Barbuda ,26',
 'North Korea,24',
 'Federated States of Micronesia,20',
 'Ethiopia,26',
 'Saint Lucia,29']

In [0]:
#rdd22=rdd2.map(lambda x:x.split(','))
rdd4=rdd2.map(lambda x: (x[0],int(x[8])))
rdd5=rdd4.reduceByKey(lambda x,y:x+y)
rdd5.take(5)


Out[18]: [('Asia', 3620036),
 ('Middle East and North Africa', 3013431),
 ('Australia and Oceania', 2111786),
 ('Central America and the Caribbean', 2698776),
 ('Europe', 6582322)]

In [0]:
from datetime import datetime
rdd6= rdd2.map(lambda x :(x[6],datetime(int(x[5]))))


In [0]:
rdd6 = rdd2.map(lambda row: row[:5] + [datetime.strptime(row[5],'%m/%d/%Y')] + row[6:])
rdd6.take(3)

Out[24]: [['Central America and the Caribbean',
  'Antigua and Barbuda ',
  'Baby Food',
  'Online',
  'M',
  datetime.datetime(2013, 12, 20, 0, 0),
  '957081544',
  '1/11/2014',
  '552',
  '255.28',
  '159.42',
  '140914.56',
  '87999.84',
  '52914.72'],
 ['Central America and the Caribbean',
  'Panama',
  'Snacks',
  'Offline',
  'C',
  datetime.datetime(2010, 7, 5, 0, 0),
  '301644504',
  '7/26/2010',
  '2167',
  '152.58',
  '97.44',
  '330640.86',
  '211152.48',
  '119488.38'],
 ['Europe',
  'Czech Republic',
  'Beverages',
  'Offline',
  'C',
  datetime.datetime(2011, 9, 12, 0, 0),
  '478051030',
  '9/29/2011',
  '4778',
  '47.45',
  '31.79',
  '226716.10',
  '151892.62',
  '74823.48']]

In [0]:
rdd6.sortBy(lambda row : row[5],ascending=False).map(lambda row : row[:5] + [row[5].strftime('%-m/%-d/%Y')] +row[6:]).take(10)


Out[25]: [['Asia',
  'Bhutan',
  'Cereal',
  'Offline',
  'M',
  '7/28/2017',
  '223854434',
  '8/25/2017',
  '2356',
  '205.70',
  '117.11',
  '484629.20',
  '275911.16',
  '208718.04'],
 ['Sub-Saharan Africa',
  'Senegal',
  'Cosmetics',
  'Online',
  'C',
  '7/26/2017',
  '537970721',
  '8/18/2017',
  '6346',
  '437.20',
  '263.33',
  '2774471.20',
  '1671092.18',
  '1103379.02'],
 ['Middle East and North Africa',
  'United Arab Emirates',
  'Household',
  'Online',
  'C',
  '7/26/2017',
  '419542396',
  '8/8/2017',
  '773',
  '668.27',
  '502.54',
  '516572.71',
  '388463.42',
  '128109.29'],
 ['Australia and Oceania',
  'Australia',
  'Beverages',
  'Online',
  'L',
  '7/26/2017',
  '631485402',
  '8/12/2017',
  '9418',
  '47.45',
  '31.79',
  '446884.10',
  '299398.22',
  '147485.88'],
 ['Sub-Saharan Africa',
  "Cote d'Ivoire",
  'Vegetables',
  'Online',
  'H',
  '7/24/2017',
  '588388097',
  '8/25/2017',
  '5968',
  '154.06',
  '90.93',
  '919430.08',
  '542670.24',
  '376759.8

In [0]:
rdd7=rdd2.map(lambda x : (x[2],x[2].count('a')))
rdd7.take(3)
rdd77=rdd7.filter(lambda x:x[1]>1).distinct()
rdd77.collect()

Out[26]: [('Personal Care', 2)]

In [0]:
data = rdd2.map(lambda row: (row[0], row[1], int(row[8])))
max_unit_sold = data.groupBy(lambda row: (row[0], row[1])).mapValues(lambda rows: max(rows, key=lambda x: x[2]))
result = max_unit_sold.groupBy(lambda row: row[1][0]).mapValues(lambda rows: max(rows, key=lambda x: x[1][2]))
result.map(lambda row: row[1][1]).collect()


Out[27]: [('Asia', 'Thailand', 9991),
 ('Australia and Oceania', 'Samoa ', 9990),
 ('Middle East and North Africa', 'Libya', 9881),
 ('Central America and the Caribbean', 'Dominica', 9996),
 ('Europe', 'United Kingdom', 9999),
 ('Sub-Saharan Africa', 'Equatorial Guinea', 9984),
 ('North America', 'Greenland', 9898)]

In [0]:
rdd8=rdd2.map(lambda row: (row[2],row[9],float(row[10]))).sortBy(lambda row: row[2] ,ascending=True).distinct()
rdd8.take(5)

Out[36]: [('Beverages', '47.45', 31.79),
 ('Clothes', '109.28', 35.84),
 ('Personal Care', '81.73', 56.67),
 ('Snacks', '152.58', 97.44),
 ('Baby Food', '255.28', 159.42)]

In [0]:
rdd9=rdd6.map(lambda row : (row[5].year,int(row[8]))).groupByKey().mapValues(lambda row: sum(row))
rdd9.take(5)

Out[32]: [(2010, 3130137),
 (2016, 3280818),
 (2014, 3214899),
 (2012, 3485045),
 (2013, 3358584)]

In [0]:
rdd10=rdd2.map(lambda row: (row[2],1)).groupByKey().mapValues(lambda row: sum(row))
rdd10.take(5)

Out[33]: [('Baby Food', 445),
 ('Snacks', 398),
 ('Cereal', 385),
 ('Clothes', 386),
 ('Cosmetics', 424)]

In [0]:

rdd11=rdd2.map(lambda row: (row[1],int(row[8]))).groupByKey().mapValues(lambda row: sum(row)).max()
rdd11

Out[34]: ('Zimbabwe', 93906)